Perusahaan peminjaman sepeda di San Fransisco memintamu untuk menganalisa durasi perjalanan customer mereka. Untuk memudahkan analisa, lakukan hal-hal berikut ini:

## **STATISTIK DESKRIPTIF**

1. Lakukan perhitungan central tendency (mean, median, dan modus).
2. Lakukan pengecekan dispersi (standar deviasi, variance, dan range).
3. Lakukan pengecekan skewness dan kurtosis.
3. Analisa apakah ada data outlier berdasarkan sebaran datanya.
4. Cari tahu berapa persentase outlier terhadap data keseluruhan.
5. Bagaimana perubahan pemusatan data sebelum dan sesudah penghapusan outlier?

## **API**

Buatlah API untuk:
- `Menampilkan` seluruh entry data setelah dilakukan handling outlier.
- `Menghapus` entry data, dari data yang telah dihandling outlier.

clue:
- Load data csv yang sudah diolah dengan pandas, kemudian konversi data ke dictionary `df.to_dict()` atau json `df.to_json()` untuk dapat diolah lebih lanjut dengan API menggunakan FastAPI.
- Pembuatan API dilakukan pada script `.py`
- Endpoint dan penamaan function pada API dibebaskan.


---



Data dapat diakses dari BigQuery dengan ketentuan:

- Project_id: `bigquery-public-data`
- Dataset: `san_francisco_bikeshare`
- Table: `bikeshare_trips`
- Ambil hanya kolom `duration_sec` saja
- Berikan LIMIT pada data yang diambil sebesar 3000 entry data

Koneksikan data di atas menggunakan `BigQuery` ke Google Colab dengan code berikut:
```py
from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
print('Authenticated')

project_id = "rock-wonder-317907" #GUNAKAN GCP PROJECT-ID KALIAN MASING-MASING
client = bigquery.Client(project=project_id)
```

Untuk melakukan Query menggunakan cara ini, kamu dapat menggunakan method `client.query('Masukkan Querynya').to_dataframe()`. Outputnya akan berupa Pandas dataframe, sehingga harus import Pandas.

Contoh:

```py
df = client.query('''
SELECT *
FROM `bigquery-public-data.thelook_ecommerce.orders`
WHERE created_at < "2022-07-01"
ORDER BY year,month ASC
''').to_dataframe()
```



In [1]:
# Working area
from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
print('Authenticated')

project_id = "kisi-kisi-lc3-442003" #GUNAKAN GCP PROJECT-ID KALIAN MASING-MASING
client = bigquery.Client(project=project_id)

Authenticated


In [2]:
df = client.query('''
SELECT duration_sec
FROM `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
LIMIT 3000
''').to_dataframe()

df

,duration_sec
0,788
1,1113
2,965
3,560
4,489
...,...
2995,1301
2996,1607
2997,892
2998,906


In [3]:
#Rata-rata
df['duration_sec'].mean()

830.2213333333333

Rata-rata duration_sec adalah **30.2213333333333**

In [4]:
#Nilai tengah
df['duration_sec'].median()

501.0

Median duration_sec adalah **501.0**

In [5]:
#Modus
df['duration_sec'].mode().values[0]

250

Modus duration_sec adalah **275**

In [6]:
#Standar Deviasi
df['duration_sec'].std()

3052.3106243840034

Standar Deviasi duration_sec adalah **3052.3106243840034**

In [7]:
#Variance
df['duration_sec'].var()

9316600.147727465

Variance duration_sec adalah **9316600.147727465**

In [8]:
#Range
range = df['duration_sec'].max() - df['duration_sec'].min()
range

67486

Range duration_sec adalah **67486** yang didapat dari nilai max durasi peminjaman dikurang nilai min durasi peminjaman

In [9]:
#skewness
df['duration_sec'].skew()

17.775518570892956

nilai skewness pada duration_sec adalah **17.775518570892956**, yang berarti distribusi yang sangat miring ke kanan yang mengindikasikan durasi peminjaman yang relatif lama yang dilakukan oleh peminjam. jadi untuk nilai outlier nya berada disisi kanan distribusi

In [10]:
#kurtosis
df['duration_sec'].kurtosis()

341.27841459621607

nilai kurtosis pada duration_sec adalah **341.27841459621607**, ini menunjukkan durasi peminjaman bersifat leptokurtic yang berarti nilai outlier lebih banyak dari distribusi normal

In [11]:
#upper dan lower boundary
q1 = df['duration_sec'].quantile(0.25)
q3 = df['duration_sec'].quantile(0.75)
iqr = q3-q1

up = q3 + 1.5*iqr
low = q1 - 1.5*iqr

print(f'Upper: {up:.2f}')
print(f'Upper: {low:.2f}')

Upper: 1514.00
Upper: -398.00


Angka upper sebesar **1514.00** memiliki arti bahwa angka melebihi outlier

Angka upper sebesar **-398.00** memiliki arti bahwa angka melebihi outlier


In [12]:
#presentasi outlier
outlier = df['duration_sec'][(df['duration_sec']<low) | (df['duration_sec']>up)]
no_outlier = df['duration_sec'][(df['duration_sec']>=low) & (df['duration_sec']<=up)]
print(f'Percentage of outliers: {len(outlier)*100/len(df["duration_sec"]):.2f}%')


Percentage of outliers: 5.43%


Percentage of outliers sebesar **5.43%** menunjukkan bahwa dari total durasi peminjaman, sekitar **5.43% di antaranya adalah outlier**, mencerminkan adanya sejumlah kecil peminjam dengan durasi peminjaman yang ekstrem.

In [13]:
#before dan after remove outlier

print("Before Extreme Value Analysis and Removal")
print('Average:',df['duration_sec'].mean())
print('Median :',df['duration_sec'].median())
print('Mode   :',df['duration_sec'].mode().values[0])

print("-"*30)

# Nilai central tendency setelah penghapusan outliers
print("After Extreme Value Analysis and Removal")
print('Average:',no_outlier.mean())
print('Median :',no_outlier.median())
print('Mode   :',no_outlier.mode().values[0])

Before Extreme Value Analysis and Removal
Average: 830.2213333333333
Median : 501.0
Mode   : 250
------------------------------
After Extreme Value Analysis and Removal
Average: 550.5174480084596
Median : 480.0
Mode   : 250


In [14]:
#menampilkan nilai outlier
outlier

,duration_sec
40,1863
41,1830
42,1623
43,2374
48,4199
...,...
2980,60239
2983,1937
2985,2787
2996,1607


In [15]:
#menampilkan nilai no outlier
no_outlier

,duration_sec
0,788
1,1113
2,965
3,560
4,489
...,...
2993,592
2994,1211
2995,1301
2997,892


In [16]:
#Export data no outlier ke csv
no_outlier.to_csv('no_outlier.csv', index=False)